# Downloading Legacy Survey DR4

This notebook describes the downloading of the 'Legacy Survey' DR4

http://legacysurvey.org/dr4/description/

There are still edge effects due to using the brick centre to check if it is in HELP. TODO: develop a nearMoc function which says if a given ra/dec is near a given moc (within a given distance). This function already exists within Stilts.

In [1]:
import subprocess

from astropy.table import Table

from pymoc import MOC

from herschelhelp_internal.utils import inMoc

In [2]:
log_file = open('log.txt', 'a')

In [50]:
#Download the list of all bricks
subprocess.call(['wget', 
                    'ftp://archive.noao.edu/public/hlsp/ls/dr4/survey-bricks-dr4.fits.gz', '-P', './data'], 
                    stdout=log_file)

#Download the list of all bricks
subprocess.call(['wget', 
                    'ftp://archive.noao.edu/public/hlsp/ls/dr4/survey-bricks.fits.gz', '-P', './data'], 
                    stdout=log_file)

#Download the list of all bricks
subprocess.call(['wget', 
                    'ftp://archive.noao.edu/public/hlsp/ls/dr3/survey-bricks-dr3.fits.gz', '-P', './data'], 
                    stdout=log_file)

0

In [5]:
dr4_bricks = Table.read('./data/survey-bricks-dr4.fits.gz')
dr3_bricks = Table.read('./data/survey-bricks-dr3.fits.gz')
total_bricks = Table.read('./data/survey-bricks.fits.gz')
print('Total bricks on whole sky: ' + str(len(total_bricks)))
print('Total bricks on DECaLS DR3: ' + str(len(dr3_bricks)))
print('Total bricks on Legacy Survey DR4: ' + str(len(dr4_bricks)))

Total bricks on whole sky: 662174
Total bricks on DECaLS DR3: 149464
Total bricks on Legacy Survey DR4: 65543


In [16]:
dr4_bricks[:10].show_in_notebook()

idx,brickname,ra,dec,nexp_g,nexp_r,nexp_z,nexphist_g [6],nexphist_r [6],nexphist_z [6],nobjs,npsf,nsimp,nexp,ndev,ncomp,psfsize_g,psfsize_r,psfsize_z,psfdepth_g,psfdepth_r,psfdepth_z,galdepth_g,galdepth_r,galdepth_z,ebv,trans_g,trans_r,trans_z,ext_g,ext_r,ext_z,wise_nobs [4],trans_wise [4],ext_w1,ext_w2,ext_w3,ext_w4
0,0556p332,55.671641791,33.25,0,0,8,0 .. 0,0 .. 0,3983213 .. 6869231,2434,1716,424,188,100,6,0.0,0.0,1.10044,0.0,0.0,23.5382,0.0,0.0,23.1585,0.498528,0.228609,0.370062,0.573473,1.60227,1.07931,0.603717,70 .. 12,0.918985 .. 0.99583,0.0917291,0.0563336,0.0120145,0.0045366
1,0557p330,55.7851239669,33.0,0,0,6,0 .. 0,0 .. 0,3515900 .. 6151845,2308,1646,381,178,99,4,0.0,0.0,1.12777,0.0,0.0,23.5938,0.0,0.0,23.2322,0.528384,0.209271,0.348674,0.554691,1.69823,1.14395,0.639873,74 .. 12,0.914347 .. 0.995581,0.0972226,0.0597073,0.012734,0.00480827
2,0557p335,55.7641196013,33.5,0,0,10,0 .. 0,0 .. 0,335689 .. 8918538,3409,2322,567,297,208,15,0.0,0.0,1.08877,0.0,0.0,23.6377,0.0,0.0,23.2558,0.55702,0.192263,0.329322,0.537254,1.79026,1.20595,0.674551,73 .. 12,0.90992 .. 0.995342,0.102492,0.0629433,0.0134242,0.0050689
3,0559p332,55.9701492537,33.25,0,0,10,0 .. 0,0 .. 0,8565 .. 11524237,3843,2595,694,332,210,12,0.0,0.0,1.10383,0.0,0.0,23.5416,0.0,0.0,23.1777,0.567837,0.186204,0.322295,0.530811,1.82503,1.22937,0.687651,72 .. 13,0.908254 .. 0.995252,0.104482,0.0641656,0.0136849,0.00516734
4,0560p330,56.0826446281,33.0,0,0,6,0 .. 0,0 .. 0,3395289 .. 6321674,2243,1566,416,159,98,4,0.0,0.0,1.15255,0.0,0.0,23.4639,0.0,0.0,23.1147,0.616166,0.161383,0.292685,0.502955,1.98036,1.334,0.746177,74 .. 12,0.900845 .. 0.994849,0.113374,0.0696267,0.0148496,0.00560712
5,0560p335,56.0631229236,33.5,0,0,10,0 .. 0,0 .. 0,60892 .. 9782339,3390,2384,590,256,145,15,0.0,0.0,1.09524,0.0,0.0,23.6286,0.0,0.0,23.2468,0.559281,0.19098,0.327841,0.535901,1.79753,1.21084,0.677289,73 .. 13,0.909572 .. 0.995323,0.102908,0.0631987,0.0134787,0.00508944
6,0562p332,56.2686567164,33.25,0,0,9,0 .. 0,0 .. 0,1228886 .. 9607509,3287,2354,528,270,128,7,0.0,0.0,1.13715,0.0,0.0,23.5552,0.0,0.0,23.2001,0.633924,0.153118,0.282502,0.493091,2.03743,1.37245,0.767682,74 .. 13,0.898138 .. 0.994701,0.116642,0.0716334,0.0152776,0.00576871
7,0563p330,56.3801652893,33.0,0,0,0,0 .. 0,0 .. 0,7450967 .. 2821842,1156,820,187,96,46,7,0.0,0.0,1.17665,0.0,0.0,23.4711,0.0,0.0,23.132,0.647852,0.146934,0.274764,0.48549,2.0822,1.4026,0.784549,73 .. 13,0.896021 .. 0.994585,0.119205,0.0732073,0.0156133,0.00589546
8,0563p335,56.3621262458,33.5,0,0,2,0 .. 0,0 .. 0,5186227 .. 4945981,1841,1402,244,119,71,5,0.0,0.0,1.12285,0.0,0.0,23.6427,0.0,0.0,23.2754,0.562955,0.188914,0.325448,0.533709,1.80934,1.2188,0.681738,74 .. 13,0.909006 .. 0.995293,0.103584,0.0636139,0.0135672,0.00512286
9,0644p677,64.4727272727,67.75,0,1,0,0 .. 0,4711324 .. 0,0 .. 0,1044,925,66,19,30,4,0.0,1.40017,0.0,0.0,22.0005,0.0,0.0,21.3913,0.0,0.516102,0.21702,0.357319,0.562342,1.65875,1.11736,0.625,113 .. 16,0.916252 .. 0.995684,0.0949628,0.0583195,0.0124381,0.00469654


In [7]:
#TODO: Maybe check for edge effects and add add extra leeway
fields= [
# Field                RAtyp    DECtyp   RAmin   RAmax   DECmin  DECmax
['AKARI-NEP',          274.65,  65.796,  263.6,  266.5,  68.48,  69.51 ],
['AKARI-SEP',          72.231, -54.380,   66.1,  75.5,  -55.96, -51.61 ],
['Bootes',             216.43,  32.401,  215.7,  220.7,  32.12,  36.18 ],
['CDFS-SWIRE',         51.022, -29.818,   50.7,  55.5,  -30.50, -25.92 ],
['COSMOS',             149.29,  1.0821,  148.6,  151.6,  0.72,   3.71  ],
['EGS',                217.27,  53.644,  212.3,  217.6,  51.08,  54.31 ],
['ELAIS-N1',           247.09,  55.175,  237.9,  247.9,  52.37,  57.59 ],
['ELAIS-N2',           248.42,  39.127,  246.1,  252.3,  39.02,  43.02 ],
['ELAIS-S1',           7.1062, -43.863,    6.3,  11.3,  -45.60, -41.53 ],
['GAMA-09',            129.07, -2.2317,  127.1,  142.2, -2.53,   3.55  ],
['GAMA-12',            172.84,  -0.482,  172.2,  187.4, -3.54,   2.59  ],
['GAMA-15',            211.75, -2.2857,  209.9,  225.3, -2.57,   3.5   ],
['HDF-N',              190.25,  62.205,  188.0,  190.4,  61.69,  62.78 ],
['Herschel-Stripe-82', 353.75, -7.1089,  348.3,  19.1,  -9.5,    9.25  ],
['Lockman-SWIRE',      161.94,  59.056,  154.7,  167.8,  54.95,  60.89 ],
['NGP',                192.89,  22.099,  189.8,  209.3,  21.62,  36.16 ],
['SA13',               197.89,  42.440,  197.5,  198.6,  42.34,  43.09 ],
['SGP',                334.29, -34.503,  333.5,  28.2,  -37.1,  -26.0  ],
['SPIRE-NEP',          266.33,  68.790,  264.6,  275.4,  64.41,  68.63 ],
['SSDF',               341.57, -59.186,  340.5,  2.8,   -61.2,  -48.6  ],
['xFLS',               261.38,  58.018,  255.6,  262.5,  57.85,  60.84 ],
['XMM-13hr',           203.31,  37.474,  202.8,  204.4,  37.31,  38.52 ],
['XMM-LSS',            32.941, -6.0229,   32.1,  38.2,  -7.53,  -1.52  ]]
   
    
#Has a field been removed?  ( RA BETWEEN 13.4 AND 36.3 AND DEC BETWEEN -2.32 AND 2.49 )
       
    
    
  

In [ ]:
help_moc = MOC()
field_mocs = []
for field in fields:
    help_moc += MOC(filename="../../dmu2/dmu2_field_coverages/{}_MOC.fits".format(field[0])) 
    field_mocs.append(MOC(filename="../../dmu2/dmu2_field_coverages/{}_MOC.fits".format(field[0])))
help_moc.write("../../dmu2/help_coverage_MOC.fits")

In [14]:
help_mask = inMoc(dr4_bricks['ra'],
                  dr4_bricks['dec'],
                  MOC(filename="../../dmu2/help_coverage_MOC.fits"))


In [22]:
print('There are ' + str(len(dr4_bricks[help_mask])) + ' bricks in Legacy Survey DR4 in HELP fields')
help_bricks_file = open("./data/help_bricks.lis", "w")
help_downloads_file = open("./data/help_bricks_downloads.lis", "w")
field_coverages = {}

for field in fields:
    
    #TODO: creplace inMoc with new function nearMoc to avoid any edge effects
    field_mask = inMoc(dr4_bricks['ra'],
                       dr4_bricks['dec'],
                       MOC(filename="../../dmu2/dmu2_field_coverages/{}_MOC.fits".format(field[0])))
    print('There are ' + str(len(dr4_bricks[field_mask])) + ' bricks in Legacy Survey DR4 for ' + field[0])
    field_coverages[field[0]] = len(dr4_bricks[field_mask])
    field_bricks_file  = open("./data/{}_bricks.lis".format(field[0]), "w")
    field_downloads_file  = open("./data/{}_bricks_downloads.lis".format(field[0]), "w")
    for brick in dr4_bricks[field_mask]:
        
        brick_string = ("public/hlsp/ls/dr4/tractor/" 
                      + brick['brickname'][:3] 
                      + '/tractor-' 
                      + brick['brickname'] 
                      + '.fits\n')
        field_bricks_file.write(brick_string)
        field_downloads_file.write("ftp://archive.noao.edu/" + brick_string)
        help_bricks_file.write(brick_string)
        help_downloads_file.write("ftp://archive.noao.edu/" + brick_string)
    field_bricks_file.close()
    field_downloads_file.close()
    

There are 1505 bricks in Legacy Survey DR4 in HELP fields
There are 0 bricks in Legacy Survey DR4 for AKARI-NEP
There are 0 bricks in Legacy Survey DR4 for AKARI-SEP
There are 175 bricks in Legacy Survey DR4 for Bootes
There are 0 bricks in Legacy Survey DR4 for CDFS-SWIRE
There are 0 bricks in Legacy Survey DR4 for COSMOS
There are 57 bricks in Legacy Survey DR4 for EGS
There are 215 bricks in Legacy Survey DR4 for ELAIS-N1
There are 147 bricks in Legacy Survey DR4 for ELAIS-N2
There are 0 bricks in Legacy Survey DR4 for ELAIS-S1
There are 0 bricks in Legacy Survey DR4 for GAMA-09
There are 0 bricks in Legacy Survey DR4 for GAMA-12
There are 0 bricks in Legacy Survey DR4 for GAMA-15
There are 0 bricks in Legacy Survey DR4 for HDF-N
There are 0 bricks in Legacy Survey DR4 for Herschel-Stripe-82
There are 182 bricks in Legacy Survey DR4 for Lockman-SWIRE
There are 618 bricks in Legacy Survey DR4 for NGP
There are 5 bricks in Legacy Survey DR4 for SA13
There are 0 bricks in Legacy Survey

## Using the RA/DEC limits
Because of edge effects it is better to use the conservative ra/dec bounding boxes

In [ ]:
print('There are ' + str(len(dr4_bricks[help_mask])) + ' bricks in Legacy Survey DR4 in HELP fields')
help_bricks_file = open("./data/help_bricks.lis", "w")
help_downloads_file = open("./data/help_bricks_downloads.lis", "w")
field_coverages = {}

def inRectangle(RAmin, RAmax, DECmin, DECmax, ra, dec):
    if RAmin < RAmax:
        if ra > RAmin and ra < RAmax and dec > DECmin and dec < DECmax:
            output = True
        else:
            output = False
    elif RAmin > RAmax:
        if (ra > RAmin or ra < RAmax) and (dec > DECmin and dec < DECmax):
            output = True
        else:
            output = False
        
    return output
        

for field in fields:
    
    #TODO: creplace inMoc with new function nearMoc to avoid any edge effects
    field_mask = inMoc(dr4_bricks['ra'],
                       dr4_bricks['dec'],
                       MOC(filename="../../dmu2/dmu2_field_coverages/{}_MOC.fits".format(field[0])))
    print('There are ' + str(len(dr4_bricks[field_mask])) + ' bricks in Legacy Survey DR4 for ' + field[0])
    field_coverages[field[0]] = len(dr4_bricks[field_mask])
    field_bricks_file  = open("./data/{}_bricks_wider.lis".format(field[0]), "w")
    field_downloads_file  = open("./data/{}_bricks_downloads_wider.lis".format(field[0]), "w")
    for brick in dr4_bricks:
        if inRectangle(field[3], field[4], field[5], field[6], brick['ra'], brick['dec'])
        
            brick_string = ("public/hlsp/ls/dr4/tractor/" 
                      + brick['brickname'][:3] 
                      + '/tractor-' 
                      + brick['brickname'] 
                      + '.fits\n')
            field_bricks_file.write(brick_string)
            field_downloads_file.write("ftp://archive.noao.edu/" + brick_string)
            help_bricks_file.write(brick_string)
            help_downloads_file.write("ftp://archive.noao.edu/" + brick_string)
    field_bricks_file.close()
    field_downloads_file.close()

## Comparison to DECaLS DR3
We were originally confused about the relation between Legacy Survey Dr4 and DECaLS DR3 and wanted to compare coverages. It seems that they only marginally overlap and that DECaLS DR3 is officially included in Legacy Survey DR4 but that the actual data provided is only from BASS and MzLS.

In [65]:
for field in fields:
    field_mask = inMoc(dr3_bricks['ra'],
                       dr3_bricks['dec'],
                       MOC(filename="../../dmu2/dmu2_field_coverages/{}_MOC.fits".format(field[0])))
    print('There are ' + str(len(dr3_bricks[field_mask])) + ' bricks in DECaLS DR3 for ' + field[0])


There are 0 bricks in AKARI-NEP
There are 0 bricks in AKARI-SEP
There are 152 bricks in Bootes
There are 0 bricks in CDFS-SWIRE
There are 83 bricks in COSMOS
There are 0 bricks in EGS
There are 0 bricks in ELAIS-N1
There are 0 bricks in ELAIS-N2
There are 0 bricks in ELAIS-S1
There are 966 bricks in GAMA-09
There are 1006 bricks in GAMA-12
There are 986 bricks in GAMA-15
There are 0 bricks in HDF-N
There are 4908 bricks in Herschel-Stripe-82
There are 0 bricks in Lockman-SWIRE
There are 2591 bricks in NGP
There are 0 bricks in SA13
There are 0 bricks in SGP
There are 0 bricks in SPIRE-NEP
There are 0 bricks in SSDF
There are 0 bricks in xFLS
There are 0 bricks in XMM-13hr
There are 349 bricks in XMM-LSS


In [ ]:
#Download only the bricks for help fields
#for field in fields
#    subprocess.call(['wget', 
#                    '-irx', '{}_bricks.lis'.format(field[0]), '-P', './data'], 
#                    stdout=log_file)

#Download just the files with HELP coverage
subprocess.call(['wget', 
                    '-irx', './data/help_bricks_downloads.lis', '-P', './data'], 
                    stdout=log_file)

In [ ]:
#Concatenate all the fits files into one fits file
for field in fields:
    if field_coverages[field[0]] == 0:
        continue
    #First creat the overall fits files by concatenating all the relevant tables
    subprocess.call(['stilts', 
                    'tcat', 'in=@{}_bricks.lis'.format(field[0]), 'out=LegacySurvey-dr4_{}.fits'.format(field[0])], 
                    stdout=log_file)
    
    #TODO: Remove all the sources which are not in HELP (Due to edge effects where part of the bricks go outside)
    #Label the points by Field
    #subprocess.call(['stilts', 
    #                'addcol', 
    #                'Field',
    #                '\"inMoc(\"../../dmu2/field_coverages/{}_MOC.fits\") ?\"COSMOS\" : \"NOFIELD\"\"'], 
    #                stdout=log_file)
    # 
    #Remove the points with "NOFIELD"
    #subprocess.call(['stilts', 
    #                'select', 
    #                 '\"Field',
    #                 '!=',
    #                 '\"NOFIELD\"\"'
    #                ], 
    #                stdout=log_file)
    
    # Then generate the MOCs
    subprocess.call(['stilts', 
                    'pixfoot', 
                     'in=LegacySurvey-dr4_{}.fits'.format(field[0]), 
                     'out=LegacySurvey-dr4_{}_MOC.fits'.format(field[0]),
                     'order=12' #Order 12 seems to avoid any holes
                     ],  
                    stdout=log_file)

In [24]:
#Can also be done with astropy
for field in fields:
    if field_coverages[field[0]] == 0:
        continue
    field_catalogue = Table.read('./data/LegacySurvey-dr4_{}.fits'.format(field[0]))
    mask = inMoc(field_catalogue['ra'],
                 field_catalogue['dec'],
                 MOC(filename='../../dmu2/dmu2_field_coverages/{}_MOC.fits'.format(field[0])))
    field_catalogue.add_column(Column(data=field[0], name="Field")) #Does it work without providing an array
    field_catalogue[mask].write('./data/LegacySurvey-dr4_{}_help_only.fits'.format(field[0]))

## Alternative method
This descibes an alternate method using the filter and tag shell script. This downloads everything and then tags it. It doesn't suffer from any edge effects but requires very large memory resources (both RAM and storage)

In [ ]:
#Download all the files (Total ~270 Gb)
subprocess.call(['wget', 
                 '-r', 
                 'ftp://archive.noao.edu/public/hlsp/ls/dr4/tractor/', 
                 '-P', 
                 './data'], 
                 stdout=log_file)

In [ ]:
#Create a list of all the fits files 
subprocess.call(['ls', 
                 './data/archive.noao.edu/public/hlsp/ls/dr4/tractor/*/*.fits', 
                 '>', 
                 '.data/tractor_files.lis'], 
                    stdout=log_file)

In [ ]:
subprocess.call(['stilts', 
                 'tcat', 
                 'in=@tractor_files.lis'.format(field[0]), 
                 'out=LegacySurvey-dr4.fits'.format(field[0])], 
                 stdout=log_file)

In [ ]:
#Use the tag script to filter in to HELP fields
subprocess.call(['./filterAndTag.sh', 
                 'LegacySurvey_dr4.fits',
                 'LegacySurvey_dr4_HELP-coverage.fits'], 
                 stdout=log_file)